Import the db_utils module containing the write_delta_v2 function

In [23]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.HR LIMIT 1000")
display(df)

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4f8e3608-ddda-4e42-98f5-e8f146d247d8)

In [24]:
%run /db_utils

StatementMeta(, , -1, Finished, Available)

In [25]:
from pyspark.sql import SparkSession

# Define database connection properties
database_host = "intechmysql.ckvdytywefte.eu-north-1.rds.amazonaws.com"
database_port = '3306'
database_name = 'intechdb'
user = 'admin'
password = 'Intech1234'
driver = "com.mysql.jdbc.Driver"
url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 33, Finished, Available)

In [26]:
# Install pymysql if not already installed (used for MySQL database connection)
!pip install pymysql
import pymysql


StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 34, Finished, Available)

DEPRECATION: notebookutils 3.3.0-20230926.8 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of notebookutils or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [27]:
# Establish a connection to the MySQL server
conn = pymysql.connect(host=database_host, user=user, password=password, database="intechdb")

# Create a cursor object to execute queries
cur = conn.cursor()

# Execute the SHOW TABLES query to fetch table names
cur.execute('SHOW TABLES')
table_names_temp = cur.fetchall()
table_names = [row[0] for row in table_names_temp]

table_names_updated = []

# Check for tables that have a 'empid' column and add them to the list
for table in table_names:
    cur.execute(f"SHOW COLUMNS FROM {database_name}.{table} LIKE 'empid'")
    column_exists = cur.fetchone() is not None
    if column_exists:
        table_names_updated.append(table)

# Print the updated table names
for table2 in table_names_updated:
    print(table2)

# Close the cursor and connection to the MySQL server
cur.close()
conn.close()

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 35, Finished, Available)

HR


In [28]:
# Set the action to either 'upsert' or 'overwrite'
# action = 'upsert'
action = 'overwrite'

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 36, Finished, Available)

In [29]:
# Iterate through the updated table names
for table_name1 in table_names_updated:
    tgt_table_name = table_name1

    # Construct the query to retrieve data from the MySQL table
    query = f"(select * from {table_name1})"
    
    # Read data from MySQL into a Spark DataFrame
    df = spark.read.format('jdbc').\
             option('url',url).\
             option('driver',driver).\
             option('user',user).\
             option('password',password).\
             option('query',query).\
             load()# .\
            #  limit(10)

    # Define the target database name
    tgt_db_name = 'tgt_db_name'
    tgt_table_name = table_name1

    # Depending on the action chosen ('upsert' or 'overwrite'), call the write_delta_v2 function
    if action == 'upsert':
        write_delta_v2(type='data_copy', df=df, mode=action, dbname=tgt_db_name, targetTable=tgt_table_name, sourceTable=tgt_table_name, keyCol='empid')
    else:
        write_delta_v2(type='data_copy', df=df, mode=action, dbname=tgt_db_name, targetTable=tgt_table_name)

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 37, Finished, Available)

table already exists!
overwrite mode


In [30]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.HR LIMIT 1000")
display(df)

StatementMeta(, 3919ceac-e4ea-4412-8e46-c3fb41e5a2a1, 38, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8bb351a9-2c64-4939-b8b4-3352dbbd2cbf)